# Airbnb - Q2 & Q3 (Boston & Seattle) - Data Modeling & Evaluation

This notebook has been created to perform the modelling part of the process and to evaluate the model after. As such it will answer the second two bussiness questions as follows:

- Can I predict prices based on the market, variables & seasonal trend?
- Which AirBnB listing variables have the biggest impact on prices?

First I will attempt to create a machine learning model to predict what prices airbnb's should charge when adding a property and then during the evaluation process I will look at feature importance. These two steps should help answer the two business questions above.

## 1. Imports

In [1]:
# Imports basic functions
import pandas as pd
import numpy as np
import re

# Imports functions for plotting graphs
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import matplotlib.style as style

# Sklearn imports
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Imports custom made airbnb package
import airbnb_pkg as airbnb

## 2. Functions

In [14]:
def plot_histogram(data, column, market, ax, binsize=50, y_label='', x_label='', color='blue'):
    """
    Function to plot standard histograms (Can be used for individual or side by side plots)
    """
    # Plots the Seattle Graph with title & axis
    sns.distplot(data[column], kde=False, color=color, bins=binsize, ax=ax)
    
    # Sets up the title based on the plotting column and the market    
    ax.set_title(f"{market} {column} data", fontsize=25)
    
    # Sets the y and x labels
    ax.set_ylabel(y_label, fontsize=20)
    ax.set_xlabel(x_label, fontsize=20)
    
    # Removes spines and changes layout to tight
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
def plot_line(data, ax, color, alpha=.7, ls='-', lw=5, label=None):
    """
    plots the standard line graph
    """
    ax.plot(data, color=color,
            alpha=alpha, ls=ls,
            linewidth=lw, label=label)
    
def outlier_removal(data, column, outlier_pctile=5):
    """
    Removes data from prices outside the 95 percentile
    """
    df = data.copy()
    upper_theshold = np.percentile(df[column].values, 100-outlier_pctile)
    lower_theshold = np.percentile(df[column].values, outlier_pctile)
    
    df = df[df[column] >= lower_theshold]
    df = df[df[column] <= upper_theshold]
    
    return df      
    
def validation_metrics(predictions, y_test, baseline_mae='', baseline_mape=''):
    # Obtain the absolute erros for each prediction
    errors = abs(predictions - test)

    print('Mean Absolute error:', round(np.mean(errors), 2))

    # Calculate mean absolute percentage error (MAPE)
    mape = np.mean(100 * (errors / test))

    print('Mean Absolute Percentage error:', round(np.mean(mape), 2))

    # Calculate and display accuracy
    accuracy = 100 - mape
    print('Accuracy:', round(accuracy, 2), '%.')
    
    if baseline_mae:
        # Compare to baseline
        improvement_baseline = mae - baseline_mae
        print('MAPE Improvement:', round(improvement_baseline, 2))    

    if baseline_mape:
        # Compare to baseline
        improvement_baseline = mape - baseline_mape
        print('MAPE Improvement:', round(improvement_baseline, 2), '%.')     

## 3. Basic Setup (Read data, create global variables etc.)

In [3]:
# Plotting settings
style.use('seaborn-poster') #sets the size of the charts
style.use('seaborn-darkgrid')
plt.rcParams['font.family'] = "sans serif"

In [4]:
# Read in the .csv files for Seattle 
Seattle_Cal = pd.read_csv('Data/Seattle/calendar.csv')
Seattle_List = pd.read_csv('Data/Seattle/listings.csv')

# Read in the .csv files for Boston 
Boston_Cal = pd.read_csv('Data/Boston/calendar.csv')
Boston_List = pd.read_csv('Data/Boston/listings.csv')

In [5]:
# Create ML input using the function from the previous notebook
Seattle_input = airbnb.ML_preprocessing(Seattle_Cal, Seattle_List)
Boston_input = airbnb.ML_preprocessing(Boston_Cal, Boston_List)

/Users/warwick.rommelrath/Udacity/Airbnb_Data_Analysis/airbnb_pkg/__init__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  for x in data['price']]
/Users/warwick.rommelrath/Udacity/Airbnb_Data_Analysis/airbnb_pkg/__init__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['date'] = pd.to_datetime(data['date'])


Columns dropped: {'square_feet'}
Number of rows dropped: 0
Columns dropped: {'square_feet'}
Number of rows dropped: 0


## 4. Modelling

To begin with I will run the modelling on the seattle data alone and then the Boston data will be ran to confirm the process:

I could apply PCA at this point but for the sake of looking into which variables are the most important I'm going to keep my data how it is (I will try PCA later on in this notebook after my initial analysis. 

For the ML I've decided to test the random forests algorthim for a few reasons:
- It's suited towards creating regression models with many features.
- It will be quick to train and get results which will be useful for a few reasons e.g. re-training on new markets, easily deployed in the workplace, keeps computational costs down.
- It's useful to look at feature importance.

The downsides potentially are:
- Although its much less prone than decision trees it can overfit occationally
- Some of the feature importance outputs may be skewed towards the catagorical variables in the data

In [6]:
# Set the X & Y variables
X = Seattle_input.iloc[:,1:]
Y = Seattle_input.iloc[:,0].values

# split into training & testing data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [11]:
# setup the random forests classifier
rf = RandomForestRegressor(oob_score=True, bootstrap=True, random_state=14)

# specify parameters for grid search
parameters = {
    'n_estimators': [1,2,10,50,100,200],
    'min_samples_leaf':[1,2,3],
    'min_samples_split': [2,3,4],
}

# create grid search object
reg = GridSearchCV(rf, param_grid=parameters, n_jobs=-1, verbose=2, cv=5)

# train classifier
reg.fit(X_train, y_train)

# predict on test data
y_pred = reg.best_estimator_.predict(X_test)

print("\nOptimised Model\n------")
print(f'RMSE Train Score: {reg.best_estimator_.score(X_train, y_train)}')
print(f'RMSE Test Score: {reg.best_estimator_.score(X_test, y_test)}')

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  4.4min finished



Optimised Model
------
RMSE Train Score: 0.9110337329607967
RMSE Test Score: 0.6097888546654754


The training accuracy score above is around RMSE  error is 91% accurate which is pretty good but the testing score of 69% is pretty bad showing that the model is acctually overfitting. One possible cause of this could be due to the amount of features we are trying to base the modelling on as we have 100+ all trying to influence it.

It's also worth pointing out that calculating the RMSE has the benefit of penalizing large errors but in this case it's possible that using MAE is a better function as it might be fine that some of the data points (the outliers) are way off from our prediction. Given this is might be worth removing these and re-testing. 

In [12]:
# Calculate accuracy based on MAE instead of RMSE

# Obtain the absolute erros for each prediction
errors = abs(y_pred - y_test)

print('Average absolute error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAE)
mae = np.mean(errors)

print('Mean Absolute error:', round(np.mean(mae), 2))

# Calculate mean absolute percentage error (MAPE)
mape = np.mean(100 * (errors / y_test))

print('Mean Absolute Percentage error:', round(np.mean(mape), 2))

# Calculate and display accuracy
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

# Save the model above as a baseline to check improvement
mae_baseline = mae
mape_baseline = mape

Average absolute error: 27.6
Mean Absolute error: 27.6
Mean Absolute Percentage error: 24.93
Accuracy: 75.07 %.


As we can see from calculating the MAE the absolute error is around 27.6 with a percentage error of 24.9% which again isn't perfect but isn't as skewed by outliers as by the RMSE. I will now remove the extreme outliers and re-create the model.

In [15]:
# Remove only the most extreme outliers
Seattle_ro = outlier_removal(Seattle_input, 'Cal_price', outlier_pctile=5)

# Set the X & Y variables
X = Seattle_ro.iloc[:,1:]
Y = Seattle_ro.iloc[:,0].values

# split into training & testing data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# setup the random forests classifier
rf = RandomForestRegressor(oob_score=True, bootstrap=True, random_state=14)

# specify parameters for grid search
parameters = {
    'n_estimators': [1,2,10,50,100,200],
    'min_samples_leaf':[1,2,3],
    'min_samples_split': [2,3,4],
}

# create grid search object
reg = GridSearchCV(rf, param_grid=parameters, n_jobs=-1, verbose=2, cv=5)

# train classifier
reg.fit(X_train, y_train)

# predict on test data
y_pred = reg.best_estimator_.predict(X_test)

print("\nOptimised Model\n------")
print(f'RMSE Train Score: {reg.best_estimator_.score(X_train, y_train)}')
print(f'RMSE Test Score: {reg.best_estimator_.score(X_test, y_test)}')

validation_metrics(y_pred, y_test, baseline_mae=mae_baseline, baseline_mape=mape_baseline)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  4.1min finished



Optimised Model
------
RMSE Train Score: 0.8701781253399095
RMSE Test Score: 0.6227280432472169
Average absolute error: 26.62
Mean Absolute error: 26.62
Mean Absolute Percentage error: 24.76
Accuracy: 75.24 %.
MAPE Improvement: -0.98
MAPE Improvement: -0.17 %.


Removing the outliers increased the accuracy slightly but the model is now probably not been as skewed by the outlier datapoints. The next step will be to perform fetaure scaling, this will ensure that variables with a bigger varience don't have a bigger influence on the data.

In [21]:
# Perform feature scaling on the input data
StdSclr = StandardScaler()
Seattle_sc = pd.DataFrame(StdSclr.fit_transform(Seattle_ro))

# Set the X & Y variables
X = Seattle_sc.iloc[:,1:]
Y = Seattle_ro.iloc[:,0].values

# split into training & testing data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# setup the random forests classifier
rf = RandomForestRegressor(oob_score=True, bootstrap=True, random_state=14)

# specify parameters for grid search
parameters = {
    'n_estimators': [1,2,10,50,100,200],
    'min_samples_leaf':[1,2,3],
    'min_samples_split': [2,3,4],
}

# create grid search object
reg = GridSearchCV(rf, param_grid=parameters, n_jobs=-1, verbose=2, cv=5)

# train classifier
reg.fit(X_train, y_train)

# predict on test data
y_pred = reg.best_estimator_.predict(X_test)

print("\nOptimised Model\n------")
print(f'RMSE Train Score: {reg.best_estimator_.score(X_train, y_train)}')
print(f'RMSE Test Score: {reg.best_estimator_.score(X_test, y_test)}')

validation_metrics(y_pred, y_test, baseline_mae=mae_baseline, baseline_mape=mape_baseline)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  4.1min finished



Optimised Model
------
RMSE Train Score: 0.9112129381772082
RMSE Test Score: 0.6041080352465447
Average absolute error: 27.28
Mean Absolute error: 27.28
Mean Absolute Percentage error: 23.3
Accuracy: 76.7 %.
MAPE Improvement: -0.32
MAPE Improvement: -1.63 %.


Fetaure scaling improved the RMSE error but acctually slightly made the MAE error slightly worse. This shows that the modelling improved on some extremes but not on the data as a whole and might suggest using a bigger outlier removal.


In order to improve the model further we should look to how each feature is contributing to the model. The best way to find which features are contributing most to the model would be to re-train the model while dropping one feature each at a time and then recomputing the model scores. This may be compuationally intestive compaired to some other methods but it will clearly show what features are driving the model and which ones are making it worse. First I will re-run the original model with the best parameters as shown below:

In [17]:
reg.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=True, random_state=14, verbose=0,
                      warm_start=False)

In [19]:
def RF_price_model(X_model_input, Y_model_input):
    """
    Creates the price regression Random Forests based on the desired input df
    (Expects dependent variable to be the first column)
    """
    # Set the X & Y variables
    X = X_model_input.iloc[:,1:]
    Y = Y_model_input.iloc[:,0].values

    # split into training & testing data
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
    # setup the random forests classifier
    rf = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=True, random_state=14, verbose=0,
                      warm_start=False)

    # train classifier
    rf.fit(X_train, y_train)

    # predict on test data
    y_pred = rf.predict(X_test)

    print("\nOptimised Model\n------")
    print(f'RMSE Train Score: {rf.score(X_train, y_train)}')
    print(f'RMSE Test Score: {rf.score(X_test,y_test)}')
    
    validation_metrics(y_pred, y_test, baseline_mae=mae_baseline, baseline_mape=mape_baseline)
    

best_model = RF_price_model(Seattle_sc, Seattle_ro)


Optimised Model
------
RMSE Train Score: 0.9126322214631966
RMSE Test Score: 0.5645713622896062
Average absolute error: 0.46
Mean Absolute error: 0.46
Mean Absolute Percentage error: -78.06
Accuracy: 178.06 %.
MAPE Improvement: -27.15
MAPE Improvement: -102.99 %.


Now we have our best model I can run it through the feature importance function I've created that loops through each column and removes it from the input prior to creating the model. If we compaire this to the accuracy of the core model we will have an idea of which columns are having a positive or negative influence:

In [ ]:
def feature_importance(model, X_train, y_train, input_data, rand_state):
    """
    This module calculates the feature importance of all columns
    """
    # Creates a list for storing the importance info
    feat_imp = []
    
    # clone the model
    mod_clone = clone(model)
    
    # set random_state, train and score the benchmark model
    mod_clone.random_state = rand_state
    mod_clone.fit(X_train, y_train)
    mod_score = mod_clone.score(X_train, y_train)
    
    # Loop through columns creating feature importance score    
    for col in X_train.columns:
        # Drop the column we are calulating the influence of         
        X_input = X_train.drop(columns=[col])
        
        # Repeat the process of cloning, random state, train and score         
        new_mod = clone(model)
        new_mod.random_state = rand_state
        new_mod.fit(X_input, y_train)
        update_score = new_mod.score(X_input, y_train)
        
        # Calulate the feature importance and append to list
        feat_imp.append(mod_score - update_score)
    
    # Create the output that shows each columns feature importance     
    importances_df = pd.DataFrame(feat_imp, index=input_data.columns)
    
    return importances_df

importance = feature_importance(best_model, X_train, y_train, Seattle_ro.iloc[:,1:], 14)

In [ ]:
def plot_feature_importance(feat_importance):
    """
    This plot is designed to plot the feature performance in the model
    """
    # It's probably easier to represent this data graphically
    fig, ax = plt.subplots(figsize=(18,9))

    # Sorts the data
    sorted_imp = feat_importance.sort_values(0, ascending=False)
    
    # Select the X & Y values
    x = range(1,len(sorted_imp)+1)
    y = sorted_imp.values

    # Creates a vertical line graph
    import seaborn as sns
    ax.vlines(x=x, ymin=0, ymax=y, color='blue', alpha=0.4)
    ax.scatter(x, y, color='blue', s=6, alpha=1)

    # Format the X-axis
    plt.xticks(range(1,len(sorted_imp)+1), list(sorted_imp.index), rotation=89, fontsize=12)

    plt.tight_layout()
    
plot_feature_importance(importance)

Its odd that the highest performing variable is 'bathrooms' some of the other high values do make sense like 'accomodates' & 'Entire home/apt'. The results show that alot of the variables that I'm putting into the model are acctually having a negative effect on the outcome and might as well be ignored.

At this point it might be worth dropping some of the values that are having a negative influence on the model or at least the worst perfoming ones and running the analysis & potentially re-running the model to see if they both improve.

In [ ]:
# Select the colums that are below zero
col_drop = list(importance[importance[0] < 0].index)

# Drop the unimportant columns
Seattle_drop = Seattle_ro.drop(columns=col_drop)

# Perform feature scaling on the input data
StdSclr = StandardScaler()
Seattle_sc_update = pd.DataFrame(StdSclr.fit_transform(Seattle_drop))

# Re-crea
new_rf = RF_price_model(Seattle_sc_update)

In [ ]:
# Re-run the feature importance
importance_updated = feature_importance(new_rf, X_train, y_train, Seattle_ro.iloc[:,1:], 14)

In [ ]:
# Re-do the graph
plot_feature_importance(importance_updated)

Although the accuracy of the model hasn't impoved it's starting to make more sense as accommodates is now top which seems more likely than bathrooms. It might be worth removing the new features that are below zero as they may help the model even more.

In [ ]:
# Select the colums that are below zero
col_drop = list(importance[importance[0] < 0].index) + list(importance_updated[importance_updated[0] < 0].index)

# Create the new inputs without these columns
Update_input = Seattle_input.drop(columns=col_drop)

# Re-crea
update_rf = RF_price_model(Seattle_input)

In [ ]:
# Re-run the feature importance
importance_updated_2 = feature_importance(rf, X_train, y_train, 14)

In [ ]:
# Re-do the graph
plot_feature_importance(importance_updated)

In [ ]:
# Make predictions on test data
predictions = best_model.predict(X_test)

# Performance metrics
errors = abs(predictions - y_test)

print('Metrics for Random Forest Trained on Expanded Data')
print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = np.mean(100 * (errors / y_test))

# Compare to baseline
improvement_baseline = 100 * abs(mape - baseline_mape) / baseline_mape
print('Improvement over baseline:', round(improvement_baseline, 2), '%.')

# Calculate and display accuracy
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')